In [1]:
from sklearn.ensemble import RandomForestClassifier
import deepchem as dc
import numpy as np
import pandas as pd
import tempfile
import chemprop
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.linear_model import LogisticRegression
import xgboost
import os
from sklearn.svm import SVC

2022-11-10 13:56:29.630774: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-10 13:56:29.730901: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-10 13:56:29.730916: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-10 13:56:29.751973: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-10 13:56:30.274246: W tensorflow/stream_executor/platform/de

In [2]:
name_all=[]
auc_roc=[]
name='all_algorithm'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','lr','dmpnn']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100


2022-11-10 13:56:34.807827: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-10 13:56:34.808333: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-11-10 13:56:34.808381: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2022-11-10 13:56:34.808420: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-11-10 13:56:34.808458: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could no

1/1 [==============================] - 0s 391ms/step - loss: 2.4509 - auc: 0.4465
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 1.8697 - auc: 0.8327
Epoch 3/100
1/1 [==============================] - 0s 34ms/step - loss: 1.9709 - auc: 0.8677
Epoch 4/100
1/1 [==============================] - 0s 27ms/step - loss: 1.9869 - auc: 0.8786
Epoch 5/100
1/1 [==============================] - 0s 33ms/step - loss: 1.8925 - auc: 0.8909
Epoch 6/100
1/1 [==============================] - 0s 31ms/step - loss: 1.7642 - auc: 0.9007
Epoch 7/100
1/1 [==============================] - 0s 32ms/step - loss: 1.5787 - auc: 0.9114
Epoch 8/100
1/1 [==============================] - 0s 32ms/step - loss: 1.4243 - auc: 0.9247
Epoch 9/100
1/1 [==============================] - 0s 28ms/step - loss: 1.3006 - auc: 0.9410
Epoch 10/100
1/1 [==============================] - 0s 32ms/step - loss: 1.2630 - auc: 0.9388
Epoch 11/100
1/1 [==============================] - 0s 28ms/step - loss: 1.2626 

1/1 [==============================] - 0s 25ms/step - loss: 0.1899 - auc: 0.9956
Epoch 89/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1810 - auc: 0.9966
Epoch 90/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1821 - auc: 0.9966
Epoch 91/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1847 - auc: 0.9958
Epoch 92/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1851 - auc: 0.9953
Epoch 93/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1857 - auc: 0.9958
Epoch 94/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1801 - auc: 0.9961
Epoch 95/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1866 - auc: 0.9955
Epoch 96/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1845 - auc: 0.9958
Epoch 97/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1834 - auc: 0.9960
Epoch 98/100
1/1 [==============================] - 0s 28ms/step - loss: 

In [3]:
name='remove_dmpnn'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','lr']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 333ms/step - loss: 1.8076 - auc: 0.6596
Epoch 2/100
1/1 [==============================] - 0s 29ms/step - loss: 1.7808 - auc: 0.8100
Epoch 3/100
1/1 [==============================] - 0s 28ms/step - loss: 1.6787 - auc: 0.8389
Epoch 4/100
1/1 [==============================] - 0s 33ms/step - loss: 1.5102 - auc: 0.8519
Epoch 5/100
1/1 [==============================] - 0s 34ms/step - loss: 1.3556 - auc: 0.8653
Epoch 6/100
1/1 [==============================] - 0s 34ms/step - loss: 1.2833 - auc: 0.8633
Epoch 7/100
1/1 [==============================] - 0s 26ms/step - loss: 1.2155 - auc: 0.8751
Epoch 8/100
1/1 [==============================] - 0s 29ms/step - loss: 1.1338 - auc: 0.9019
Epoch 9/100
1/1 [==============================] - 0s 29ms/step - loss: 1.0605 - auc: 0.9239
Epoch 10/100
1/1 [==============================] - 0s 29ms/step - loss: 0.9877 - auc: 0.9447
Epoch 11/100
1/1 [==============================] - 0s 27ms/step - l

1/1 [==============================] - 0s 28ms/step - loss: 0.2147 - auc: 0.9936
Epoch 89/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2161 - auc: 0.9928
Epoch 90/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2075 - auc: 0.9945
Epoch 91/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2091 - auc: 0.9945
Epoch 92/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2119 - auc: 0.9937
Epoch 93/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1991 - auc: 0.9958
Epoch 94/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2098 - auc: 0.9943
Epoch 95/100
1/1 [==============================] - 0s 22ms/step - loss: 0.2119 - auc: 0.9936
Epoch 96/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2132 - auc: 0.9939
Epoch 97/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2084 - auc: 0.9952
Epoch 98/100
1/1 [==============================] - 0s 23ms/step - loss: 

In [4]:
name='remove_lr'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','dmpnn']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 334ms/step - loss: 1.7486 - auc: 0.7203
Epoch 2/100
1/1 [==============================] - 0s 28ms/step - loss: 1.7977 - auc: 0.8182
Epoch 3/100
1/1 [==============================] - 0s 27ms/step - loss: 1.5876 - auc: 0.8382
Epoch 4/100
1/1 [==============================] - 0s 32ms/step - loss: 1.4143 - auc: 0.8507
Epoch 5/100
1/1 [==============================] - 0s 36ms/step - loss: 1.3406 - auc: 0.8377
Epoch 6/100
1/1 [==============================] - 0s 31ms/step - loss: 1.2763 - auc: 0.8487
Epoch 7/100
1/1 [==============================] - 0s 29ms/step - loss: 1.1742 - auc: 0.8912
Epoch 8/100
1/1 [==============================] - 0s 36ms/step - loss: 1.0777 - auc: 0.9261
Epoch 9/100
1/1 [==============================] - 0s 27ms/step - loss: 1.0045 - auc: 0.9459
Epoch 10/100
1/1 [==============================] - 0s 27ms/step - loss: 0.9691 - auc: 0.9511
Epoch 11/100
1/1 [==============================] - 0s 29ms/step - l

1/1 [==============================] - 0s 30ms/step - loss: 0.2170 - auc: 0.9927
Epoch 89/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2204 - auc: 0.9924
Epoch 90/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2174 - auc: 0.9923
Epoch 91/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2214 - auc: 0.9928
Epoch 92/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2170 - auc: 0.9926
Epoch 93/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2173 - auc: 0.9928
Epoch 94/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2311 - auc: 0.9904
Epoch 95/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2232 - auc: 0.9917
Epoch 96/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2285 - auc: 0.9899
Epoch 97/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2252 - auc: 0.9914
Epoch 98/100
1/1 [==============================] - 0s 23ms/step - loss: 

In [5]:
name='remove_fcnn'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','lr','dmpnn']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 332ms/step - loss: 2.1974 - auc: 0.4223
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 1.6472 - auc: 0.7888
Epoch 3/100
1/1 [==============================] - 0s 28ms/step - loss: 1.6582 - auc: 0.8510
Epoch 4/100
1/1 [==============================] - 0s 26ms/step - loss: 1.6834 - auc: 0.8696
Epoch 5/100
1/1 [==============================] - 0s 29ms/step - loss: 1.6602 - auc: 0.8790
Epoch 6/100
1/1 [==============================] - 0s 33ms/step - loss: 1.5389 - auc: 0.8931
Epoch 7/100
1/1 [==============================] - 0s 29ms/step - loss: 1.3848 - auc: 0.9031
Epoch 8/100
1/1 [==============================] - 0s 31ms/step - loss: 1.2330 - auc: 0.9185
Epoch 9/100
1/1 [==============================] - 0s 32ms/step - loss: 1.1383 - auc: 0.9232
Epoch 10/100
1/1 [==============================] - 0s 35ms/step - loss: 1.0570 - auc: 0.9355
Epoch 11/100
1/1 [==============================] - 0s 33ms/step - l

1/1 [==============================] - 0s 24ms/step - loss: 0.1935 - auc: 0.9954
Epoch 89/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1960 - auc: 0.9956
Epoch 90/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1987 - auc: 0.9948
Epoch 91/100
1/1 [==============================] - 0s 33ms/step - loss: 0.1864 - auc: 0.9965
Epoch 92/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1959 - auc: 0.9947
Epoch 93/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1975 - auc: 0.9947
Epoch 94/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1874 - auc: 0.9955
Epoch 95/100
1/1 [==============================] - 0s 23ms/step - loss: 0.1909 - auc: 0.9958
Epoch 96/100
1/1 [==============================] - 0s 30ms/step - loss: 0.1899 - auc: 0.9961
Epoch 97/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1952 - auc: 0.9945
Epoch 98/100
1/1 [==============================] - 0s 27ms/step - loss: 

In [6]:
name='remove_xgb'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['fcnn','lr','dmpnn']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 334ms/step - loss: 3.8217 - auc: 0.2175
Epoch 2/100
1/1 [==============================] - 0s 26ms/step - loss: 2.1420 - auc: 0.4608
Epoch 3/100
1/1 [==============================] - 0s 24ms/step - loss: 1.5243 - auc: 0.8286
Epoch 4/100
1/1 [==============================] - 0s 23ms/step - loss: 1.6163 - auc: 0.8636
Epoch 5/100
1/1 [==============================] - 0s 34ms/step - loss: 1.7504 - auc: 0.8753
Epoch 6/100
1/1 [==============================] - 0s 34ms/step - loss: 1.8351 - auc: 0.8816
Epoch 7/100
1/1 [==============================] - 0s 28ms/step - loss: 1.9298 - auc: 0.8708
Epoch 8/100
1/1 [==============================] - 0s 29ms/step - loss: 1.8955 - auc: 0.8746
Epoch 9/100
1/1 [==============================] - 0s 26ms/step - loss: 1.8203 - auc: 0.8839
Epoch 10/100
1/1 [==============================] - 0s 31ms/step - loss: 1.6722 - auc: 0.8959
Epoch 11/100
1/1 [==============================] - 0s 30ms/step - l

1/1 [==============================] - 0s 28ms/step - loss: 0.2776 - auc: 0.9884
Epoch 89/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2717 - auc: 0.9892
Epoch 90/100
1/1 [==============================] - 0s 22ms/step - loss: 0.2772 - auc: 0.9876
Epoch 91/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2670 - auc: 0.9893
Epoch 92/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2800 - auc: 0.9867
Epoch 93/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2679 - auc: 0.9889
Epoch 94/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2568 - auc: 0.9911
Epoch 95/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2677 - auc: 0.9892
Epoch 96/100
1/1 [==============================] - 0s 22ms/step - loss: 0.2639 - auc: 0.9896
Epoch 97/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2637 - auc: 0.9893
Epoch 98/100
1/1 [==============================] - 0s 26ms/step - loss: 

In [7]:
de=pd.DataFrame()
de['name']=name_all
de['auc_roc']=auc_roc
de.to_csv('4model_remove_one_model.csv',index=False)
print(de)

            name   auc_roc
0  all_algorithm  0.945005
1   remove_dmpnn  0.941485
2      remove_lr  0.939427
3    remove_fcnn  0.944644
4     remove_xgb  0.940599
